### Notebook for constructiong POD-NODE NIROM approximation for a 2D Burgers' example

A collection of high-fidelity snapshots are generated that sufficiently capture the time-dynamics of the simulation. POD is adopted to define a reduced basis space for the high-fidelity snaphosts. The evolution of the time dynamics in the POD-latent space is modeled using Neural ODEs (NODE).  

Firedrake is used as the high-fidelity model for solving the 2D viscous Burgers equation to generate snapshots


In [ ]:
### Loading modules
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import scipy
import os
import gc
import argparse
import platform
print("Python "+str(platform.python_version()))
import importlib
from importlib import reload as reload

import tensorflow as tf
print("Tensorflow "+ str(tf.__version__))
if tf.__version__ == '1.15.0':
    tf.compat.v1.enable_eager_execution()
elif tf.__version__.split('.')[0] == 2: 
    print("Setting Keras backend datatype")
    tf.keras.backend.set_floatx('float64')

from tfdiffeq import odeint,odeint_adjoint
from tfdiffeq.adjoint import odeint as adjoint_odeint
tf.keras.backend.set_floatx('float64')
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
np.random.seed(0)

basedir   = os.getcwd()
srcdir    = os.path.join(basedir,'../pynirom/')
workdir   = os.path.join(basedir,'../examples/')
datadir   = os.path.join(basedir,'../data/')
figdir    = os.path.join(basedir,'../figures/podnode')
nodedir   = os.path.join(basedir,'../data/')
savedir = nodedir+'burgers2d/current'

if not os.path.exists(savedir):
    os.makedirs(savedir)
if not os.path.exists(figdir):
    os.makedirs(figdir)

import pynirom
from pynirom.pod import pod_utils as pod
from pynirom.rbf import plotting as rpu
from pynirom.utils import data_utils as du
from pynirom.node import main as nd
from pynirom.node import plotting as pu
from pynirom.node import node as node



In [ ]:
device = 'cpu:0'           # select gpu:# or cpu:#
purpose= 'train'           #'train' to train a new model, 
                           # 'retrain' to start training from an existing model, and 
                           # 'eval' to load a pre-trained model for evaluation 
pre_trained_dir = savedir+'/model_weights/' #If 'eval' specify path for pretrained model
stacking = True            #If True, Specify new stacking order of latent space vector
stack_order = 'v_x,v_y'    #If stacking = True, specify the stacking order of the latent space vector
scale_time = True          #Scale time or not (Normalize)
scale_states = False       #Scale states or not 
scaling_method = 'centered' #Scaling method: 'centered', 'minmax' or 'maxabs'
augmented,aug_dims = (False,5) #Augmented or not and size of augmentation
N_layers = int(1)          #Only four layers supported as of now.
N_neurons = int(40)        #Number of neurons per layer
act_f = 'tanh'             #Activation Function ('linear', 'tanh', 'sigmoid',...), default='tanh'
learning_rate_decay = True #Use decaying learning rate or not
initial_learning_rate = float(0.01) #If 'learning_rate_decay = False' then this is the fixed learning rate
decay_steps = int(401)     #Number of steps for learning rate decay
decay_rate = float(0.9)    #Rate of learning rate decay
staircase_opt = True       #True for staircase decay and False for exponential
optimizer = 'RMSprop'      #See pynirom.node.node.set_optimizer() for options
use_adjoint = False        #Use adjoint method or not
solver = 'euler'           #Specify ODE solver. See tfdiffeq README for available options 
use_minibatch, batch_size = (False,256) #Use minibatch or not and batch size
epochs = int(2000)         #Number of epochs of training


print("\n***** Runtime parameters: ******\n")
print(f'Mode = {purpose}, Scaling = {scale_states}, Augmenting = {augmented}, Adjoint = {use_adjoint}')
print(f'Solver = {solver}, Optimizer = {optimizer}, Stacking order = {stack_order}, Epochs = {epochs}')
print(f'# Layers = {N_layers}, # Neurons per layer = {N_neurons}, Activation fn = {act_f}')
if use_minibatch:
    print(f'Use minibatch = {use_minibatch}, Batch size = {batch_size}')
if learning_rate_decay:
    print(f'Init LR = {initial_learning_rate}, # LR decay steps = {decay_steps}, LR decay rate = {decay_rate}')
else:
    print(f'Fixed LR = {initial_learning_rate}')
print('**********************************\n')



In [ ]:
### ------ Import Snapshot data -------------------
data = np.load(datadir + 'Burgers2D_Re1000_Nn3721_Nt1500.npz')

print('HFM data has {0} snapshots of dimension {1} for p,u and v, spanning times [{2}, {3}]'.format(
                    data['time'].shape[0],data['v_x'].shape[0],
                    data['time'][0], data['time'][-1]))


## ------- Prepare training snapshots ----------------
print('\n-------Prepare training and testing data---------')
soln_names = ['v_x', 'v_y']
nodes = data['nodes'];  
triangles = data['elems']; 
snap_start = 10
T_end = 1500.0        ### 5 seconds
snap_incr = 4

snap_train, times_train = du.prepare_data(data, soln_names, start_skip=snap_start, T_end=T_end, incr=snap_incr)
print('Using {0} training snapshots for time interval [{1},{2}] seconds'.format(times_train.shape[0],
                                        times_train[0], times_train[-1]))

## ------- Prepare testing snapshots ----------------
pred_incr = snap_incr - 3
snap_pred_true, times_predict = du.prepare_data(data, soln_names, start_skip=snap_start, incr=pred_incr)
print('Using {0} testing snapshots for time interval [{1},{2}] seconds'.format(times_predict.shape[0],
                                        times_predict[0], times_predict[-1]))


del data
gc.collect()

In [ ]:
### ------ Compute the POD basis using the training snapshots------------------
trunc_lvl = 0.99
snap_norm, snap_mean, U, D, W = pod.compute_pod_multicomponent(snap_train)
nw, U_r = pod.compute_trunc_basis(D, U, eng_cap = trunc_lvl)

### ------ Compute the POD coefficients for training snapshots------------------
Z_train = pod.project_onto_basis(snap_train, U_r, snap_mean)


### ------ Compute the POD coefficients for the truth snapshots on the prediction interval------------------
Z_pred_true = pod.project_onto_basis(snap_pred_true, U_r, snap_mean)

npod_total = 0
for key in soln_names:
    npod_total+=nw[key]
    
    

In [ ]:
rpu.plot_sing_val(D)


In [ ]:
### ---- Setup NODE input data

NODE = nd.NODEBase(device=device)
true_state_array, true_pred_state_array, init_state, state_len, dt_train, dt_predict = \
        NODE.prepare_input_data(Z_train, nw, times_train, stack_order, times_predict, Z_pred_true)

print("Training NODE using %d modes for %d time steps with %.3f <= t <= %.3f and dt = %.3f"%(state_len,
                             true_state_array.shape[0], times_train[0], 
                             times_train[-1], dt_train))
print("Predicting NODE solutions using %d modes for %d time steps with %.3f <= t <= %.3f and dt = %.3f"%(
                            state_len, true_pred_state_array.shape[0], times_predict[0], 
                            times_predict[-1], dt_predict))


In [ ]:
### Preprocess training data (scale time and/or states, augment states if using ANODE)
### Set up learning rate scheduler and optimizer for training of the NODE model

true_state_tensor, times_tensor, init_tensor, learn_rate, optim = \
                NODE.preprocess_data(scale_states=scale_states, scale_time=scale_time, augmented=augmented, 
                        lr_decay=learning_rate_decay, init_lr=initial_learning_rate, opt=optimizer, 
                        scaling_method=scaling_method, aug_dim=aug_dims, 
                        decay_steps=decay_steps, decay_rate=decay_rate, staircase=staircase_opt, )



In [ ]:
%%time 

### ---- Model Training ------
train_loss_results, train_lr, saved_ep = \
NODE.train_model(true_state_tensor, times_tensor, init_tensor, epochs, savedir, \
                 solver=solver, purpose=purpose, adjoint=use_adjoint, \
                 minibatch=use_minibatch, pre_trained_dir = pre_trained_dir)


In [ ]:
## --- Generate NODE predictions ---

predicted_states, times_predict = NODE.predict_time(times_predict, init_tensor, pre_trained_dir,)

## ---- Compute Mean Square Error of predictions
Z_pred = {}
ctr= 0
for key in stack_order.split(','):
    Z_pred[key] = np.array(predicted_states)[:,ctr:ctr+nw[key]].T
    ctr += nw[key]
snap_pred = pod.reconstruct_from_rom(Z_pred, U_r, snap_mean, nw)


error_vx = np.mean(np.square(snap_pred['v_x']-snap_pred_true['v_x']))
error_vy = np.mean(np.square(snap_pred['v_y']-snap_pred_true['v_y']))

print("\n---- Mean Square Error of NODE predictions ----\n")
print('Vx MSE: ' + str(error_vx))
print('Vy MSE: ' + str(error_vy))

In [ ]:
def set_label(key):
    if key == 'v_x':
        return 'u'
    elif key == 'v_y':
        return 'v'


In [ ]:
## ----- Visualize true and predicted POD coefficients -------

comp = 0
Nc = len(soln_names)
# Visualization fluff here
fig, ax = plt.subplots(nrows=Nc,ncols=3,figsize=(15,10))
mnum = comp
for iy, key in enumerate(soln_names):
    for ix in range(3):
        tt = ax[iy,ix].plot(times_predict[:],true_pred_state_array[:,mnum+ix],label='True',marker='o',
                        markevery=100, markersize=8)
        # Visualization of modal evolution using NODE
        ln, = ax[iy,ix].plot(times_predict[:],predicted_states[:,mnum+ix],label='PODNODE',color='orange',marker='D',
                        markevery=150, markersize=8)    

        ax[iy,ix].set_xlabel('Time (seconds)', fontsize=18)
        sv = set_label(key)+', mode '+str(comp+ix)
        ax[iy,ix].set_ylabel(sv,fontsize=18)
        ax[iy,ix].legend(fontsize=14)
    mnum = mnum + nw[key]
fig.suptitle("POD coefficients of the HFM and PODNODE solutions for 2D Burgers' equations", fontsize=20)
fig.tight_layout(rect=[0, 0.03, 1, 0.99])

# os.chdir(figdir)
# plt.savefig('burgers_podnode_coeff_tskip%d_oskip%d_epochs%d'%(snap_incr,pred_incr,epochs),bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(6,5))
plt.semilogy(range(epochs), train_loss_results, label='Training')
if use_minibatch:
    plt.semilogy(range(epochs), val_loss_results,label='Validation')
plt.legend(fontsize=16); plt.xticks(fontsize=14); plt.yticks(fontsize=14)
plt.ylabel('$\log$ | MSE Loss |',fontsize=16); plt.xlabel('epochs',fontsize=16)

# os.chdir(figdir)
# plt.savefig('burgers_podnode_train_loss_epochs%d.pdf'%(epochs),bbox_inches='tight')

In [ ]:
## Visualize PODNODE and HFM solutions

ky = soln_names[0]
itime=767
vmin = np.minimum(snap_pred[ky][:,itime].min(),snap_pred_true[ky][:,itime].min())
vmax = np.maximum(snap_pred[ky][:,itime].max(),snap_pred_true[ky][:,itime].max())

fig,axs = plt.subplots(1, 2, figsize=(10,6), constrained_layout=True)
cf = axs[0].tripcolor(nodes[:, 0], nodes[:, 1],triangles, snap_pred[ky][:,itime], 
                   cmap=plt.cm.jet, shading='gouraud', vmin=vmin, vmax=vmax)
axs[0].axis('equal'); axs[0].axis('off')
axs[0].set_title('PODNODE $\mathbf{%s}$ solution\n at t=%.2f seconds'%(set_label(ky),times_predict[itime]))


cf = axs[1].tripcolor(nodes[:, 0], nodes[:, 1],triangles, snap_pred_true[ky][:,itime], 
                   cmap=plt.cm.jet, shading='gouraud', vmin=vmin, vmax=vmax)
axs[1].axis('equal'); axs[1].axis('off')
axs[1].set_title('HFM $\mathbf{%s}$ solution\n at t=%.2f seconds'%(set_label(ky),times_predict[itime]))
fig.colorbar(cf, ax=axs[:], shrink=0.7, location='bottom', pad=0.04, aspect=30)

# os.chdir(figdir)
# plt.savefig('burgers_podnode_%s_t%.2f_tskip%d_oskip%d.pdf'%(ky,times_predict[itime],snap_incr,pred_incr),bbox_inches='tight')


In [ ]:
### Visualize solution error
fig = plt.figure(figsize=(8,6))
plt.tripcolor(nodes[:, 0], nodes[:, 1],triangles, 
                    snap_pred[ky][:,itime]-snap_pred_true[ky][:,itime], 
                    cmap=plt.cm.jet, shading='flat')
plt.axis('equal'); plt.axis('off')
# rpu.viz_err(snap_pred[ky][:,itime],snap_pred_true[ky][:,itime],nodes,triangles)

plt.title('PODNODE $\mathbf{%s}$ error\n at t=%.2f'%(ky,times_predict[itime]))
plt.colorbar(shrink=0.9, aspect=20, pad=0.05)

# os.chdir(figdir)
# plt.savefig('burgers_podnode_%s_err_t%.2f_tskip%d_oskip%d.pdf'%(ky,times_predict[itime],snap_incr,pred_incr),bbox_inches='tight')

In [ ]:
## ---- Compute spatial RMS/Relative error
reload(nd)
reload(pu)
metric = 'rel'
err = NODE.compute_error(snap_pred_true, snap_pred, soln_names, metric=metric)

vstring = {}
for key in soln_names:
    vstring[key] = set_label(key)

In [ ]:
ky1 = soln_names[0]; ky2 = soln_names[1]; 
t_unit = 'seconds'
tseries = times_predict
freq = tseries.size//20

fig = plt.figure(figsize=(16,4))
ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(tseries[:], err[ky1][:], 'r-s', markersize=8,
                label='$\mathbf{%s}$'%(vstring[ky1]),lw=2, markevery=freq)
ymax_ax1 = err[ky1][:].max()
ax1.set_xlabel('Time (%s)'%t_unit);lg=plt.legend(ncol=2, fancybox=True,)

ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(tseries[:], err[ky2][:], 'b-o', markersize=8,
                label='$\mathbf{%s}$'%(vstring[ky2]), lw=2, markevery=freq)
ymax_ax2 = err[ky2][:].max()
ax2.set_xlabel('Time (%s)'%t_unit);lg=plt.legend(ncol=2, fancybox=True,)
if metric == 'rms':
    fig.suptitle('Spatial RMS errors of PODNODE NIROM solutions for 2D Burgers example',fontsize=18)
elif metric == 'rel':
    fig.suptitle('Relative errors of PODNODE NIROM solutions for 2D Burgers example',fontsize=18)

In [ ]:
#### ----- Save predicted solutions -------
save_nirom_solutions = False

if save_nirom_solutions:
    os.chdir(nodedir)
    print("Saving results in %s"%(os.getcwd()))

    np.savez_compressed('burgers2d_online_node', 
                        v_x=snap_pred['v_x'], v_y=snap_pred['v_y'],
                        time=times_predict, loss=train_loss_results)
